<a href="https://colab.research.google.com/github/matheusz17/Sistema-banc-rio/blob/main/Projeto_Banco_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3  # Biblioteca que usamos pra trabalhar com o banco de dados SQLite

# Fazendo a conexão com o banco de dados (se o arquivo não existir, ele cria automaticamente)
conn = sqlite3.connect("sistema_bancario.db")
cursor = conn.cursor()

# Criação das tabelas que vamos usar para guardar as contas e o histórico de transações
cursor.execute("""
CREATE TABLE IF NOT EXISTS contas (
    numero_conta INTEGER PRIMARY KEY,
    nome_cliente TEXT NOT NULL,
    saldo REAL NOT NULL
)
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS historico (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    numero_conta INTEGER NOT NULL,
    operacao TEXT NOT NULL,
    valor REAL NOT NULL,
    FOREIGN KEY (numero_conta) REFERENCES contas(numero_conta)
)
""")

conn.commit()  # Não podemos esquecer de salvar tudo no banco de dados

# Função pra criar uma conta nova no banco
def criar_conta():
    nome = input("Digite o nome do cliente: ")
    numero_conta = int(input("Digite o número da conta (único): "))
    saldo_inicial = float(input("Digite o saldo inicial: "))

    # Aqui a gente verifica se a conta já existe no banco
    cursor.execute("SELECT * FROM contas WHERE numero_conta = ?", (numero_conta,))
    if cursor.fetchone():  # Se encontrar uma conta com esse número, avisa o erro
        print("Número de conta já existe! Tente novamente.")
        return

    # Se não existir, a conta é criada e registrada no banco
    cursor.execute("INSERT INTO contas (numero_conta, nome_cliente, saldo) VALUES (?, ?, ?)",
                   (numero_conta, nome, saldo_inicial))
    conn.commit()
    print("Conta criada com sucesso!")

# Função pra consultar o saldo de uma conta
def consultar_saldo():
    numero_conta = int(input("Digite o número da conta: "))

    # Verifica se a conta existe e pega o saldo
    cursor.execute("SELECT saldo FROM contas WHERE numero_conta = ?", (numero_conta,))
    conta = cursor.fetchone()
    if conta:
        print(f"Saldo atual: R$ {conta[0]:.2f}")  # Exibe o saldo formatado
    else:
        print("Conta não encontrada!")  # Caso não ache a conta, mostra o erro

# Função pra depositar dinheiro em uma conta
def depositar():
    numero_conta = int(input("Digite o número da conta: "))
    valor = float(input("Digite o valor a ser depositado: "))

    # Verifica se a conta existe
    cursor.execute("SELECT saldo FROM contas WHERE numero_conta = ?", (numero_conta,))
    conta = cursor.fetchone()
    if conta:
        novo_saldo = conta[0] + valor  # Atualiza o saldo
        # Atualiza o saldo no banco de dados
        cursor.execute("UPDATE contas SET saldo = ? WHERE numero_conta = ?", (novo_saldo, numero_conta))
        cursor.execute("INSERT INTO historico (numero_conta, operacao, valor) VALUES (?, 'Depósito', ?)",
                       (numero_conta, valor))  # Registra a transação no histórico
        conn.commit()
        print(f"Depósito realizado com sucesso! Novo saldo: R$ {novo_saldo:.2f}")
    else:
        print("Conta não encontrada!")  # Caso a conta não exista, mostra o erro

# Função pra sacar dinheiro de uma conta
def sacar():
    numero_conta = int(input("Digite o número da conta: "))
    valor = float(input("Digite o valor a ser sacado: "))

    # Verifica se a conta existe
    cursor.execute("SELECT saldo FROM contas WHERE numero_conta = ?", (numero_conta,))
    conta = cursor.fetchone()
    if conta:
        if conta[0] >= valor:  # Verifica se o saldo é suficiente
            novo_saldo = conta[0] - valor  # Atualiza o saldo após o saque
            cursor.execute("UPDATE contas SET saldo = ? WHERE numero_conta = ?", (novo_saldo, numero_conta))
            cursor.execute("INSERT INTO historico (numero_conta, operacao, valor) VALUES (?, 'Saque', ?)",
                           (numero_conta, valor))  # Registra a transação no histórico
            conn.commit()
            print(f"Saque realizado com sucesso! Novo saldo: R$ {novo_saldo:.2f}")
        else:
            print("Saldo insuficiente!")  # Caso não tenha saldo suficiente, avisa o erro
    else:
        print("Conta não encontrada!")  # Se a conta não existir, mostra o erro

# Função pra encerrar uma conta
def encerrar_conta():
    numero_conta = int(input("Digite o número da conta: "))

    # Verifica se a conta existe e se o saldo é zero
    cursor.execute("SELECT saldo FROM contas WHERE numero_conta = ?", (numero_conta,))
    conta = cursor.fetchone()
    if conta:
        if conta[0] == 0:  # Só pode encerrar a conta se o saldo for zero
            cursor.execute("DELETE FROM contas WHERE numero_conta = ?", (numero_conta,))
            conn.commit()
            print("Conta encerrada com sucesso!")
        else:
            print("Não é possível encerrar a conta. O saldo deve ser zero!")  # Se o saldo não for zero, não deixa encerrar
    else:
        print("Conta não encontrada!")  # Se a conta não existir, mostra o erro

# Função que exibe o menu do sistema
def menu():
    while True:
        print("\n--- Sistema Bancário ---")
        print("1. Criar conta")
        print("2. Consultar saldo")
        print("3. Depositar")
        print("4. Sacar")
        print("5. Encerrar conta")
        print("6. Sair")

        opcao = input("Escolha uma opção: ")

        # Chama a função correspondente à opção escolhida
        if opcao == "1":
            criar_conta()
        elif opcao == "2":
            consultar_saldo()
        elif opcao == "3":
            depositar()
        elif opcao == "4":
            sacar()
        elif opcao == "5":
            encerrar_conta()
        elif opcao == "6":
            print("Saindo do sistema...")  # Se escolher sair, sai do sistema
            break
        else:
            print("Opção inválida! Tente novamente.")  # Caso a opção seja inválida, pede pra tentar novamente

# Executa o menu
menu()

# Fecha a conexão com o banco de dados quando terminar
conn.close()



--- Sistema Bancário ---
1. Criar conta
2. Consultar saldo
3. Depositar
4. Sacar
5. Encerrar conta
6. Sair


KeyboardInterrupt: Interrupted by user